# StenoTexty

Tento notebook analyzuje stenografické přepisy schůzí PS ČR.

## Instalace knihovny Parlamentikon

Nastavení notebooku pro prostředí Google Colab a pro lokální běh.

In [1]:
# Specifické příkazy pro prostředí Google Colab
if 'google.colab' in str(get_ipython()):
        import os, sys
        os.chdir('/content')
        # Stažení knihovny
        ! ls parlamentikon || git clone "https://github.com/parlamentikon/parlamentikon.git" --branch main
        os.chdir('/content/parlamentikon/notebooks')

In [2]:
instalace_zavislosti = True
if instalace_zavislosti:
    ! pip install -r ../requirements.txt 1>/dev/null

You should consider upgrading via the '/home/borjana/.venv/finance/bin/python -m pip install --upgrade pip' command.


In [3]:
instalace_knihovny = False
if instalace_knihovny:
    ! pip install ..  1>/dev/null
else:
    # Přidání cesty pro lokální import knihovny
    import sys, os
    sys.path.insert(0, os.path.abspath('..'))

## Import knihoven

In [186]:
from datetime import datetime, timedelta
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random

In [5]:
from parlamentikon.Hlasovani import Organy
from parlamentikon.Snemovna import *
from parlamentikon.StenoTexty import *
from parlamentikon.utility import *

In [6]:
from nastav_notebook import nastav_pandas
nastav_pandas()

## Nastavení parametrů

In [7]:
# Data se budou pokaždé znovu stahovat z achivu PS
stahni=True

## Načtení dat

In [8]:
vsechny_organy = Organy(volebni_obdobi=-1)
snemovny = vsechny_organy[vsechny_organy.nazev_typ_organ_cz == 'Parlament'].od_organ.dt.year.sort_values()
snemovny = snemovny.sort_values().to_list()
posledni_snemovna = snemovny[-1]
assert isinstance(posledni_snemovna, int)
print(f"Snemovny dle počátku roku: {snemovny}.")
print(f"Poslední sněmovna: {posledni_snemovna}.")

2021-06-06:12:38:46 INFO     [utility.py:21] Stahuji 'https://www.psp.cz/eknih/cdrom/opendata/poslanci.zip'.


Snemovny dle počátku roku: [1992, 1996, 1998, 2002, 2006, 2010, 2013, 2017].
Poslední sněmovna: 2017.


### Načtení tabulku StenoTexty

In [9]:
st = StenoTexty(volebni_obdobi=2017, stahni=True)

2021-06-06:12:38:47 INFO     [utility.py:21] Stahuji 'https://www.psp.cz/eknih/cdrom/opendata/poslanci.zip'.
2021-06-06:12:38:48 WARNING  [Snemovna.py:149] While merging 'funkce' with 'typ_funkce': Dropping ['nazev_typ_organ_en__typ_funkce', 'typ_id_typ_organ__typ_funkce', 'nazev_typ_organ_cz__typ_funkce', 'id_typ_organ__typ_funkce', 'typ_organ_obecny__typ_funkce'] because of abundance.
2021-06-06:12:38:48 INFO     [utility.py:21] Stahuji 'https://www.psp.cz/eknih/cdrom/opendata/steno.zip'.
2021-06-06:12:38:52 INFO     [StenoTexty.py:45] Stahuji stenografické přepisy pro volební období 2017. Proces může trvat v řádu desítek minut.
2021-06-06:12:38:52 INFO     [TabulkyStenoTexty.py:148] K stažení: 11259 souborů.
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    3.2s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    8.6s
[Parallel(n

## Analýza: otázky z poslední schůze sněmovny

In [187]:
def vyber_otazky(text):
    result = re.split('(?<=[.!?]) +', text)
    if len(result) > 0:
        result = [r for r in result if len(r) > 0 and r[-1] == '?']
    else:
        result = []
    return result

posledni_schuze = st.schuze.max()
posledni_den = st.date.max()
otazky_posledni_schuze = st[st.schuze == posledni_schuze].text.apply(lambda x: vyber_otazky(x)).to_list()
#otazky_posledni_schuze = st[(st.schuze == posledni_schuze) & (st.date == posledni_den)].text.apply(lambda x: vyber_otazky(x)).to_list()
otazky_posledni_schuze = [v for o in otazky_posledni_schuze for v in o]
random.shuffle(otazky_posledni_schuze)
print(f"Bylo nalezeno {len(otazky_posledni_schuze)} otázek.")
print(f"Zobrazuji náhodných 20 otázek:")
otazky_posledni_schuze[:20]

Bylo nalezeno 252 otázek.
Zobrazuji náhodných 20 otázek:


['To je úroveň představy, se kterou se do toho šlo?',
 'Podíval jste se na strukturu těch dotací?',
 'Neúspěchy vlády?',
 'Takže co je pravdou?',
 'A pokud tyto podniky necháme zkrachovat, kdo bude našemu státu oporou v případě krize?',
 'Když říkáme: A máte napsané ty změny, pane ministře?',
 'Pamatujete si na to ještě?',
 'Vzpomenete si, milá vládo, na to, jak jste nakupovali pomůcky z Číny, za respirátory FFP3 platili částku od 60 do 422 korun, v případě FFP2 dokonce 777 korun?',
 'Kde máte nějakou představu, co budete dělat?',
 'A kolik se letos zprovozňuje?',
 'A víte, kolik se zahajuje letos?',
 'Jaký je plán?',
 'Dámy a pánové, opravdu věříte této vládě?',
 'Přijde vám to normální?',
 'Tak pro koho asi bude pracovat takový člověk?',
 'To je docela dlouhá doba na to něco předvést, nemyslíte?',
 'Říká: čau lidi, už vás štvu?',
 'Je to jen kvůli volebním preferencím, nebo chcete aktivně posloužit cizí propagandě?',
 'Nebo jak je to přísloví?',
 'Vidíte někde ty rychlodráhy, které A

In [194]:
otazky_posledni_schuze_df = pd.DataFrame({"otazka": otazky_posledni_schuze})
otazky_posledni_schuze_df['schuze'] = posledni_schuze
otazky_posledni_schuze_df.head()

,otazka,schuze
0,"To je úroveň představy, se kterou se do toho šlo?",106
1,Podíval jste se na strukturu těch dotací?,106
2,Neúspěchy vlády?,106
3,Takže co je pravdou?,106
4,"A pokud tyto podniky necháme zkrachovat, kdo bude našemu státu oporou v případě krize?",106


## Export dat

In [195]:
# Data bez extrapolace
! mkdir -p "../docs/data"
export_path = "../docs/data/otazky_posledni_schuze_PS_CR.csv"
otazky_posledni_schuze_df[['otazka', 'schuze']].to_csv(export_path)
! ls -l {export_path}

-rw-rw-r-- 1 borjana borjana 17863 čen  6 16:55 ../docs/data/otazky_posledni_schuze_PS_CR.csv


## Export dat

In [196]:
print(f"Poslední běh notebooku dokončen {datetime.now().strftime('%d.%m.%Y v %H:%M:%S')}.")

Poslední běh notebooku dokončen 06.06.2021 v 16:56:03.
